In [ ]:
import pandas as pd

In [ ]:
data_path = '/home/godwin/Documents/Workflow/Churn-Prediction-in-a-Telecom-Company/data/Telco-Customer-Churn.csv'
data = pd.read_csv(data_path)

In [ ]:
import os
path = '/home/godwin/Documents/Workflow/Churn-Prediction-in-a-Telecom-Company/mlruns/1/62d11921be9a43ca8fe1fdae4478eb24/artifacts/model/.'

if os.path.exists(path):
    print(True)

In [ ]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date in dd/month/year format
formatted_date = now.strftime("%d/%B/%Y")


In [ ]:
import urllib

model_name = "Custormer-churn-models"
model_stage="Production"
# loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_stage}")
path = f"models:/{model_name}/{model_stage}"

def split_models_uri(uri):
        """
        Split 'models:/<name>/<version>/path/to/model' into
        ('models:/<name>/<version>', 'path/to/model').
        """
        path = urllib.parse.urlparse(uri).path
        if path.count("/") >= 3 and not path.endswith("/"):
            splits = path.split("/", 3)
            model_name_and_version = splits[:3]
            artifact_path = splits[-1]
            return "models:" + "/".join(model_name_and_version), artifact_path
        return uri, ""

In [ ]:
split_models_uri(path)

In [ ]:
import mlflow

mlflow.get_registry_uri()

sqlite:////app/mlflow.db

In [ ]:
parsed = urllib.parse.urlparse(path, allow_fragments=False)
path = parsed.path
parts = path.lstrip("/").split("/")
parts

In [ ]:
from mlflow import MlflowClient

client = MlflowClient(registry_uri=mlflow.get_registry_uri())
(name, version) = (model_name, model_stage)
download_uri = client.get_model_version_download_uri(name, version)

In [ ]:
client = pymongo.MongoClient("mongodb://127.0.0.1:27018")
db = client.get_database('prediction_service')
collection = db.get_collection("data")

In [ ]:
import time
import pandas as pd
from pymongo import MongoClient

# # Connect to MongoDB
# client = MongoClient('localhost', 27017)
# dbname = 'CustomerAnalytics'
# dbcollection = 'CustomerProfiles'

# def load_data_todb(path, dbname, dbcollection):

    
#     db = client[dbname]
#     collection = db[dbcollection]

#     # Load CSV file into DataFrame
#     df = pd.read_csv(path)
#     df['log_time'] = time.time()

#     # Convert DataFrame to dictionary
#     data = df.to_dict(orient='records')

#     # Insert data into MongoDB collection
#     collection.insert_many(data)

#     print("Data imported successfully!")
    


In [ ]:
data_path = '../../data/churn-data/data/raw_data/Telco-Customer-Churn.csv'
# load_data_todb(data_path, dbname, dbcollection)

In [ ]:
client = MongoClient('localhost', 27017)
dbname = client['CustomerAnalytics']
dbcollection = dbname['CustomerProfiles']

data = [p for p in dbcollection.find()]

In [ ]:
import time
import sqlite3

conn = sqlite3.connect("CustomerAnalytics.db")
cursor = conn.cursor()


tablename = "CustomerProfile"
def create_df_table(dbname, tablename, dfpath):

    conn = sqlite3.connect(dbname)

    df = pd.read_csv(dfpath)
    df['log_time'] = time.time()
    
    df.to_sql(tablename, conn, if_exists='fail', index=False)
    conn.close()

def insert_df(dbname, dfpath, tablename):

    conn = sqlite3.connect(dbname)
    df = pd.read_csv(dfpath)
    df['log_time'] = time.time()

    df.to_sql(tablename, conn, if_exists='append', index=False)
    conn.close()

def insert_record(dbname, tablename, record: tuple):

    conn = sqlite3.connect(dbname)
    cursor = conn.cursor()
    cursor.execute(f"INSERT INTO {tablename} {(record)}")

    conn.commit()
    conn.close()

def load_data(dbname, tablename, filter_str = None):

    conn = sqlite3.connect(dbname)
    query = f"SELECT * FROM {tablename}"

    # Load data into a DataFrame
    df = pd.read_sql(query, conn)

    return df


In [ ]:
create_df_table("CustomerAnalytics.db", tablename, data_path)

In [ ]:
df.head()

In [ ]:
def create_database_table(datapath, dbname, tablename):

    conn = sqlite3.connect(dbname)
    cursor = conn.cursor()

    df = pd.read_csv(datapath)
    df['log_time'] = time.time()


    column_dict = {}

    categorical_col = df.dtypes[df.dtypes == "object"].index.tolist()
    categorical_col = {col:"text"  for col in categorical_col}
    column_dict.update(categorical_col)

    int_col = df.dtypes[df.dtypes == "int64"].index.tolist()
    int_col = {col: "int" for col in int_col}
    column_dict.update(int_col)

    float_col = df.dtypes[df.dtypes == "float64"].index.tolist()
    float_col = {col: "real" for col in float_col}
    column_dict.update(float_col)

    columns_str = ", ".join([f"{col_name} {data_type}" for col_name, data_type in column_dict.items()])

    cursor.execute(f"CREATE TABLE {tablename} ({columns_str})")

    conn.commit()
    conn.close()


def update_data(tablename, df):

    conn = sqlite3.connect(dbname)
    cursor = conn.cursor()

    data = df.to_dict(orient='records')
    cursor.execute()



## MLFLOW

In [ ]:
import pandas as pd
from prefect import task, flow
from sklearn.model_selection import train_test_split
from model import train_LR, train_DT, train_RF, train_XGB
from utils import PROCESSED_DATASET


# Load Data
@task
def load_data(path):

    data = pd.read_csv(path)
    y = data["churn"]
    X = data.drop(["churn"], axis=1)
    return (X, y)


@task
def split_data(data):

    X, y = data
    X = X.to_dict(orient="record")
    (train_x, test_x, train_y, test_y) = train_test_split(
        X, y, test_size=0.3, random_state=1993
    )
    return (train_x, test_x, train_y, test_y)


@flow(name="Training and Model Evaluation")
def main():
    # Load the processed dataset and split into train and test sets
    data = load_data(PROCESSED_DATASET)
    data = split_data(data)

    # Train the model and get the evaluation results on the training set
    print("Training Linear Regression..")
    lr_best_params = train_LR(data)
    print("Successfully Trained Linear Regression")
    print()

    print("Training Decision Tree..")
    dt_best_params = train_DT(data)
    print("Successfully Trained Decision Tree")
    print()

    print("Training Random Forest..")
    rf_best_params = train_RF(data)
    print("Successfully Trained Random Forest")
    print()

    print("Training Xgboost..")
    xgb_best_params = train_XGB(data)
    print("Successfully Trained Xgboost")
    print()

    print("Best Linear Regression Parameters")
    print(lr_best_params)
    print()
    print("=========================================================================")

    print("Best Decision Tree Parameters")
    print(dt_best_params)
    print()
    print("=========================================================================")

    print("Best Random Forest Parameters")
    print(rf_best_params)
    print()
    print("=========================================================================")

    print("Best Xgboost Parameters")
    print(xgb_best_params)
    print()
    print("=========================================================================")


if __name__ == "__main__":
    main()


In [ ]:
import os
import requests

PARENT_DIR = os.path.dirname(os.getcwd())
EVIDENTLY_SERVICE_ADDRESS = os.getenv("EVIDENTLY_SERVICE", "http://127.0.0.1:5000")
MONGODB_ADDRESS = os.getenv("MONGODB_ADDRESS", "mongodb://127.0.0.1:27017")

PROCESSED_DATASET = "../../data/processed_data/churn.csv"


MLFLOW_TRACKING_URI = f"sqlite:///{PARENT_DIR}/notebooks/mlflow.db"

# MLFLOW_TRACKING_URI = f"sqlite:///{parent_directory}/model/mlflow.db"


# def save_to_db(record, prediction, collection):
#     rec = record.copy()
#     rec["prediction"] = prediction
#     collection.insert_one(rec)


# def send_to_evidently_service(record, prediction):
#     rec = record.copy()
#     rec["prediction"] = prediction
#     requests.post(f"{EVIDENTLY_SERVICE_ADDRESS}/iterate/churn", json=[rec])


In [ ]:
import os

os.path.join("/home/databases/", "CustomerProflie.db")